<img width="8%" alt="LinkedIn.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/LinkedIn.png" style="border-radius: 15%">

# LinkedIn - Send like to latest company post
<a href="https://bit.ly/3JyWIk6">Give Feedback</a> | <a href="https://github.com/jupyter-naas/awesome-notebooks/issues/new?assignees=&labels=bug&template=bug_report.md&title=LinkedIn+-+Send+like+to+latest+company+post:+Error+short+description">Bug report</a>

**Tags:** #linkedin #socialmedia #like #company #post #python

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel)

**Last update:** 2023-08-23 (Created: 2023-08-03)

**Description:** This notebook will follow a company on LinkedIn and send like to its last posts. The post URL will be hashed using SHA and stored in a directory. This ensures that you don't engage with the same post multiple times.


<div class="alert alert-info" role="info" style="margin: 10px">
<b>Disclaimer:</b><br>
This code is in no way affiliated with, authorized, maintained, sponsored or endorsed by Linkedin or any of its affiliates or subsidiaries. It uses an independent and unofficial API. Use at your own risk.

This project violates Linkedin's User Agreement Section 8.2, and because of this, Linkedin may (and will) temporarily or permanently ban your account. We are not responsible for your account being banned.
<br>
</div>

## Input

### Import libraries

In [ ]:
from naas_drivers import linkedin
import requests
import naas
import os
try:
    import hashlib
except:
    !pip install hashlib --user
    import hashlib

### Setup variables
**Mandatory**

[Learn how to get your cookies on LinkedIn](https://www.notion.so/LinkedIn-driver-Get-your-cookies-d20a8e7e508e42af8a5b52e33f3dba75)
- `li_at`: Cookie used to authenticate Members and API clients.
- `JSESSIONID`: Cookie used for Cross Site Request Forgery (CSRF) protection and URL signature validation.
- `urls`: This variable represents a list of LinkedIn companies URL.
- `cron`: This variable represents the CRON syntax used to run the scheduler. More information here: https://crontab.guru/#0_12,18_*_*_1-5

**Optional**

- `reaction_type`: This variable represents the type of reaction to sent to the post. It could be "LIKE", "PRAISE", "APPRECIATION", "EMPATHY", "INTEREST", "ENTERTAINMENT"
- `dir_path`: This variable represents the directory path to store the post URL.

In [ ]:
# Mandatory
li_at = naas.secret.get("LINKEDIN_LI_AT") or "YOUR_LINKEDIN_LI_AT" #example: AQFAzQN_PLPR4wAAAXc-FCKmgiMit5FLdY1af3-2
JSESSIONID = naas.secret.get("LINKEDIN_JSESSIONID") or "YOUR_LINKEDIN_JSESSIONID" #example: ajax:8379907400220387585
urls = [
    "https://www.linkedin.com/company/naas-ai/",
]
cron = "0 12,18 * * 1-5" #At minute 0 past hour 12 and 18 on every day-of-week from Monday through Friday.

# Optional
reaction_type = "LIKE" # "PRAISE", "APPRECIATION", "EMPATHY", "INTEREST", "ENTERTAINMENT"
dir_path = "posts_liked"

## Model

### Connect to LinkedIn

In [ ]:
LK = linkedin.connect(li_at, JSESSIONID)
cookies = LK.cookies
headers = LK.headers

### Send like to post

In [ ]:
def send_like(
    cookies,
    headers,
    post_url,
    reaction_like
):
    # Check post URL
    response = None
    if not post_url.startswith("https://www.linkedin.com/"):
        print("🛑 Post URL not valid! Please update it in the input section.")
        return response
    # Parse url to get activity id
    if ":activity:" in post_url:
        activity_tag = ":activity:"
        tag_end = "?"
    elif "-activity-" in post_url:
        activity_tag = "-activity-"
        tag_end = "-"
    activity_id = post_url.split(activity_tag)[-1].split(tag_end)[0]
    
    # Send like
    url = f"https://www.linkedin.com/voyager/api/voyagerSocialDashReactions?threadUrn=urn%3Ali%3Aactivity%3A{activity_id}"
    payload = {"reactionType": reaction_like}
    res = requests.post(url, headers=headers, cookies=cookies, json=payload)
    if res:
        if res.status_code == 201:
            print("👍 Like sent to:", post_url)
        else:
            print(res.json())
    return res

## Output

### Send like to latest posts published

In [ ]:
def create_sha_256_hash(message):
    # Encode the message to bytes
    message_bytes = message.encode()

    # Create the hash object
    sha_256_hash = hashlib.sha256(message_bytes)

    # Return the hexadecimal digest of the hash
    return sha_256_hash.hexdigest()

def is_file_exists(dir_path, file_name):
    file_path = os.path.join(dir_path, file_name)
    try:
        with open(file_path):
            return True
    except IOError:
        return False

def save_file(dir_path, file_name):
    # Create dir
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    
    # Create file path
    file_path = os.path.join(dir_path, file_name)
    
    # Open the file in write mode. This will create the file if it doesn't exist.
    with open(file_path, 'w') as f:
        pass  # Do nothing

# Loop on URL
for url in urls:
    # Get last post
    df = LK.company.get_posts_feed(url, limit=1)
    if len(df) > 0:
        # Get post URL
        post_url = df.loc[0, "POST_URL"]
        
        # Transform URL to SHA
        sha = create_sha_256_hash(post_url)
        
        # Check if file exists
        if not is_file_exists(dir_path, sha):
            # Send like
            send_like(
                cookies,
                headers,
                post_url,
                reaction_type
            )
            
            # Save post URL as SHA
            save_file(dir_path, sha)
        else:
            print("👌 You already liked this post: ", post_url)

### Add scheduler

In [ ]:
naas.scheduler.add(cron=cron)

# Delete scheduler
# naas.scheduler.delete()